# Welcome to FIX!
In this notebook, we give some examples of **loading dataset** and **running benchmarks**.

# Part 1: Loading Datasets

We'll first talk about how to load the datasets described in our paper.
FIX is built using the `exlib` library, which we load using a local version for now. You can uncomment the `!pip install exlib` line and comment out the `import sys; sys.path.insert(0, "../src")` line if you do not have a local version you are testing.

In [1]:
# !pip install exlib
import exlib
import torch
import torch.nn.functional as F
import torchvision
import datasets as huggingface_datasets # Not to be confused with exlib.datasets!
import matplotlib.pyplot as plt
import numpy as np

/home/antonxue/lib/miniconda3/envs/exlib_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'timm'

## Cholecystectomy Dataset

This dataset contains image data from cholecystectomy surgery (gallbladder removal). The fields are as follows:

* `image`: A view of the surgery.
* `gonogo`: Whether it is safe or unsafe to operate. Background (0), safe (1), and unsafe (2).
* `organs`: Relevant organ structures for surgery. Background (0), liver (1), gallbladder (2), and hepatocystic triangle (3). These are the expert-specified interpretable features.

You can access the dataset [here on Hugging Face](https://huggingface.co/datasets/BrachioLab/cholecystectomy).
For more information, see this [notebook example](https://colab.research.google.com/github/BrachioLab/exlib/blob/main/notebooks/fix/cholec.ipynb).

In [ ]:
cholec_dataset = exlib.datasets.cholec.CholecDataset(split="test")
cholec_item = cholec_dataset[0]
cholec_image = cholec_item["image"]
cholec_gonogo = cholec_item["gonogo"]
cholec_organs = cholec_item["organs"]

print("image:", cholec_image.shape, cholec_image.dtype)
print("gonogo:", cholec_gonogo.shape, cholec_gonogo.dtype)
print("organs:", cholec_organs.shape, cholec_organs.dtype)

In [ ]:
plt.clf()
fig, ax = plt.subplots(1, 3, figsize=(12,4))
for i in range(3): ax[i].set_axis_off()

cholec_image = torchvision.transforms.GaussianBlur(41, (50.0, 50.0))(cholec_image)

ax[0].imshow(cholec_image.numpy().transpose(1,2,0))
ax[1].imshow(cholec_gonogo.numpy())
ax[2].imshow(cholec_organs.numpy())

ax[0].set_title("Original Image (blurred)")
ax[1].set_title("Back (0), Safe (1), Unsafe (2)")
ax[2].set_title("Back (0), Liver (1), Gallb. (2), Hept. (3)")

## Chest X-ray Dataset

This dataset contains vision data for chest X-ray pathology identification.
The fields are as follows:

* `image`: The image of the chest X-ray.
* `pathols`: A binary vector that denotes which of the 14 pathologies are present.
* `struct`: A collection of binary masks over the image for the relevant anatomical structures. These are the expert-specified interpretable features.

You can access the dataset [here on Hugging Face](https://huggingface.co/datasets/BrachioLab/chestx).
For more information, see this [notebook example](https://colab.research.google.com/github/BrachioLab/exlib/blob/main/notebooks/fix/chestx.ipynb).

In [ ]:
chestx_dataset = exlib.datasets.chestx.ChestXDataset(split="test")

# Find an image that has at least 2 pathologies present
for i in range(10000):
    chestx_item = chestx_dataset[i]
    if chestx_item["pathols"].sum() > 1: break

chestx_image = chestx_item["image"]
chestx_pathols = chestx_item["pathols"]
chestx_structs = chestx_item["structs"]

print("image:", chestx_image.shape, chestx_image.dtype)
print("pathols:", chestx_pathols.shape, chestx_pathols.dtype)
print("structs:", chestx_structs.shape, chestx_structs.dtype)

In [ ]:
print("All pathologies:")
print(", ".join([f"({i}) {s}" for i,s in enumerate(chestx_dataset.pathology_names)]))

print("\nAll structures:")
print(", ".join([f"({i}) {s}" for i,s in enumerate(chestx_dataset.structure_names)]))

In [ ]:
plt.clf()
fig, ax = plt.subplot_mosaic([
    (["image"] + [f"struct{i}" for i in range(7)]),
    (["."] + [f"struct{i}" for i in range(7,14)]),
], figsize=(14,4))

for _, a in ax.items(): a.set_axis_off()

struct_titles = chestx_dataset.structure_names
ax["image"].imshow(chestx_image.numpy().transpose(1,2,0), cmap="gray")
ax["image"].set_title("Image")

for i in range(14):
    mask_t = chestx_structs[i].unsqueeze(0)
    ax[f"struct{i}"].imshow((mask_t.numpy().transpose(1,2,0)) * 2, cmap="gray")
    ax[f"struct{i}"].set_title(f"{struct_titles[i][:10]} (T)", fontsize=10)

In [ ]:
print(f"Pathologies present:")
for idx in chestx_pathols.nonzero():
    print("*", chestx_dataset.pathology_names[idx.item()])

## Cosmological Mass Maps Dataset

This dataset contains clean simulated weak lensing maps without noise.
The relevant fields are as follows:
* `input`: A (1,66,66)-shaped weak lensing map
* `label`: A pair of numbers that represents the cosmological parameters Omega_m and sigma_8. In this dataset, the expert-specified features are absent.

You can access the dataset [here on Hugging Face](https://huggingface.co/datasets/BrachioLab/massmaps-cosmogrid-100k).
For more information, see this [notebook example](https://colab.research.google.com/github/BrachioLab/exlib/blob/main/notebooks/fix/mass_maps.ipynb).

In [ ]:
# massmaps_dataset = huggingface_datasets.load_dataset(exlib.datasets.mass_maps.DATASET_REPO, split="test")
massmaps_dataset = exlib.datasets.mass_maps.MassMapsDataset(split="test")

In [ ]:
# Plot a few examples of mass maps
plt.clf()
fig, ax = plt.subplots(1,4, figsize=(16,4))
for i in range(len(ax)):
    mm_input = massmaps_dataset[i]["input"]
    mm_label = massmaps_dataset[i]["label"]
    ax[i].imshow(mm_input.numpy().transpose(1,2,0))
    ax[i].set_axis_off()
    ax[i].set_title(f"Omega_m = {mm_label[0]:.3f},  sigma_8 = {mm_label[1]:.3f}")   

## Emotion Dataset

This dataset contains 58k carefully curated Reddit comments labeled for 27 emotion categories or Neutral. The fields are as follows:
* `text`: The reddit comment.
* `labels`: The emotion annotations.
* `comment_id`: Unique identifier of the comment.

You can access the dataset [here on Hugging Face](https://huggingface.co/datasets/BrachioLab/emotion).
For more information, see this [notebook example](https://colab.research.google.com/github/BrachioLab/exlib/blob/main/notebooks/fix/emotion.ipynb).

In [ ]:
emotion_dataset = exlib.datasets.emotion.EmotionDataset(split="test")

In [ ]:
emotion_dataloader = torch.utils.data.DataLoader(emotion_dataset, batch_size=4, shuffle=False)
emotion_model = exlib.datasets.emotion.EmotionClassifier().eval()

for batch in emotion_dataloader: 
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    output = emotion_model(input_ids, attention_mask)
    utterances = [
        emotion_dataset.tokenizer.decode(input_id, skip_special_tokens=True)
        for input_id in input_ids
    ]
    for utterance, label in zip(utterances, output.logits):
        id_str = emotion_model.model.config.id2label[label.argmax().item()]
        print("Text: {}\nEmotion: {}\n".format(utterance, id_str))
    break

## Multilingual Politeness Dataset

This dataset contains conversation snippets from Wikipedia's editor talk pages. The fields are as follows:
* `text`: The Wikipedia's editor talk page conversation snippets.
* `politeness`: politeness level from -2 (very rude) to 2 (very polite).

You can access the dataset [here on Hugging Face](https://huggingface.co/datasets/BrachioLab/multilingual_politeness).
For more information, see this [notebook example](https://colab.research.google.com/github/BrachioLab/exlib/blob/main/notebooks/fix/multilingual_politeness.ipynb).

In [ ]:
politeness_dataset = exlib.datasets.multilingual_politeness.PolitenessDataset(split="test")

In [ ]:
politeness_dataloader = torch.utils.data.DataLoader(politeness_dataset, batch_size=4, shuffle=False)
politeness_model = exlib.datasets.multilingual_politeness.PolitenessClassifier().eval()

for batch in politeness_dataloader: 
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    output = politeness_model(input_ids, attention_mask)
    utterances = [
        politeness_dataset.tokenizer.decode(input_id, skip_special_tokens=True)
        for input_id in input_ids
    ]
    for utterance, label in zip(utterances, output):
        print("Text: {}\nPoliteness: {}\n".format(utterance, label.item()))
    break

## Supernova Dataset

This dataset contains astronomical time-series that has 18 types of astronomical sources. The fields are as follows:
* `label`: The class of the object.
* `times_wv`: 2D array of shape containing observation times (modified Julian days, MJD) and filter (wavelength) for each observation.
* `target`: 2D array of shape containing the flux (arbitrary units) and flux error for each observation.

You can access the dataset [here on Hugging Face](https://huggingface.co/datasets/BrachioLab/supernova-timeseries).
For more information, see this [notebook example](https://colab.research.google.com/github/BrachioLab/exlib/blob/main/notebooks/fix/supernova.ipynb).

In [ ]:
supernova_dataset = exlib.datasets.supernova.SupernovaDataset(split="test")

In [ ]:
supernova_item = supernova_dataset.dataset[0]
times_wv = torch.tensor(supernova_item["times_wv"])
xs = times_wv[:, 0]
ys = torch.tensor(supernova_item["target"])[:, 0]
times_wv, xs, ys = times_wv[ys!=0], xs[ys!=0], ys[ys!=0]
unique_wls = [3670.69, 4826.85, 6223.24, 7545.98, 8590.9, 9710.28]

In [ ]:
plt.clf()
plt.figure(figsize=(6,4))

for wl in unique_wls:
    mask = times_wv[:, 1] == wl
    plt.scatter(xs[mask], ys[mask], label=f'{wl:.2f}', cmap='viridis')

plt.title(f'Class: {supernova_item["label"]}')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.legend(title='Wavelength')

# Part 2: Running benchmarks and evaluations

We now show how to run some benchmarks + evaluations.


## Vision Example (Cholecystectomy)

We show an example of groups generated by the quickshift segmentation algorithm compared to those that are labeled by our surgeon experts.

In [ ]:
cholec_dataset = exlib.datasets.cholec.CholecDataset(split="test")
cholec_item = cholec_dataset[0]
cholec_image = cholec_item["image"]
cholec_organs = cholec_item["organs"]

quickshift_feature_extractor = exlib.features.vision.QuickshiftGroups(max_groups=8)
quickshift_groups = quickshift_feature_extractor(cholec_image.unsqueeze(0))[0]

# Image, expert-specified groups, and quickshift-generated groups
cholec_image.shape, cholec_organs.shape, quickshift_groups.shape

In [ ]:
# Let's visualize these things
plt.clf()
fig, ax = plt.subplots(1, 3, figsize=(12,4))
for a in ax: a.set_axis_off()

cholec_image = torchvision.transforms.GaussianBlur(41, (50.0, 50.0))(cholec_image)

ax[0].imshow(cholec_image.numpy().transpose(1,2,0))
ax[1].imshow(cholec_item["organs"].numpy())
ax[2].imshow((torch.arange(8).view(-1,1,1) * quickshift_groups).sum(dim=0).numpy())

ax[0].set_title("Original image (blurred)")
ax[1].set_title("Expert-specified groups")
ax[2].set_title("Quickshift-generated groups")

In [ ]:
# Expects a pair of (N,C,H,W)-shaped things
cholec_metric = exlib.datasets.cholec.CholecFixScore()

# Adjust things to be able to be put into cholec_metric
cholec_organs_mask = F.one_hot(cholec_organs).permute(2,0,1).unsqueeze(0) # (1,num_true_groups,H,W)
batched_qs_groups = quickshift_groups.unsqueeze(0) # (1,num_pred_groups,H,W)

# The FIX score is computed for each pixel, and we just take the average
cholec_fix_score = cholec_metric(groups_pred=batched_qs_groups, groups_true=cholec_organs_mask)
print(cholec_fix_score.shape, cholec_fix_score.mean())

## Natural Language Example (Multilingual Politeness)

We show an example of the groups demarcated at the sentence level.

In [ ]:
politeness_dataset = exlib.datasets.multilingual_politeness.PolitenessDataset(split="test")
politeness_item = politeness_dataset[0]
politeness_word_list = [w for w in politeness_item["word_list"] if w.strip()]

sentence_feature_extractor = exlib.features.text.SentenceGroups(distinct=26, scaling=1.5)
sentence_groups = sentence_feature_extractor(politeness_word_list)

In [ ]:
print(" ".join(politeness_word_list), "\n")
for i, g in enumerate(sentence_groups):
    sentence = " ".join([politeness_word_list[ni.item()] for ni in g.nonzero()])
    print(f"Sentence {i+1}: {sentence}")

## Time-series Example (Supernova)

We show an example of groups based on slicing.

In [ ]:
supernova_dataset = exlib.datasets.supernova.SupernovaDataset(split="test")
supernova_item = supernova_dataset.dataset[0]
times_wv = torch.tensor(supernova_item["times_wv"])
xs = times_wv[:, 0]
ys = torch.tensor(supernova_item["target"])[:, 0]
times_wv, xs, ys = times_wv[ys!=0], xs[ys!=0], ys[ys!=0]
unique_wls = [3670.69, 4826.85, 6223.24, 7545.98, 8590.9, 9710.28]

slice_feature_extractor = exlib.features.time_series.SliceGroups(ngroups=10, window_size=100)

In [ ]:
supernova_dataloader = exlib.datasets.supernova_helper.create_test_dataloader_raw(
    dataset = supernova_dataset,
    batch_size = 5
)

for batch in supernova_dataloader:
    slice_groups = slice_feature_extractor(**batch)
    slice_groups = slice_groups[0]
    break

In [ ]:
plt.clf()
plt.figure(figsize=(6,4))

# First plot the wavelengths like before
for wl in unique_wls:
    mask = times_wv[:, 1] == wl
    plt.scatter(xs[mask], ys[mask], label=f'{wl:.2f}', cmap="viridis")

# Then overlay the groups
cmap = plt.cm.get_cmap("viridis")
for i, g in enumerate(slice_groups):
    if g.sum() == 0: continue
    xmin = xs[g.nonzero().min().item()].item()
    xmax = xs[g.nonzero().max().item()].item()
    plt.axvspan(xmin, xmax, alpha=0.3, facecolor=cmap(i/len(slice_groups)))

plt.title(f'Class: {supernova_item["label"]}')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.legend(title='Wavelength')

# Part 3: Get scores for a list of baselines

We now show how to get FixScore for a list of feature extractors for their corresponding datasets.
This part can be run standalone from the previous parts.

In [ ]:
# !pip install exlib
import sys; sys.path.insert(0, "../src")
import exlib
import torch
import torch.nn.functional as F
import torchvision
import datasets as huggingface_datasets # Not to be confused with exlib.datasets!
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import exlib.datasets as xd
import exlib.features.vision as xfv
import exlib.features.time_series as xfts
import exlib.features.text as xft
from datasets import load_dataset

# there is a preprocess_xx in each folder

l = [
    # dataset, feature_extractor, metric, preprocess (can be 
    (xd.cholec.CholecDataset(split="test"), xfv.QuickshiftGroups(max_groups=8), xd.cholec.CholecFixScore(), xd.cholec.preprocess_cholec),
    (xd.chestx.ChestXDataset(split="test"), xfv.QuickshiftGroups(max_groups=20), xd.chestx.ChestXFixScore(), xd.chestx.preprocess_chestx),
    (xd.mass_maps.MassMapsDataset(split="test"), xfv.QuickshiftGroups(kernel_size=5, 
    max_dist=10, sigma=0.2, max_groups=25), xd.mass_maps.MassMapsFixScore(), xd.mass_maps.preprocess_mass_maps),
    (xd.supernova.SupernovaDataset(split="test"), xfts.SliceGroups(ngroups=5, window_size=100), xd.supernova.SupernovaFixScore(),
    xd.supernova.preprocess_supernova),
    (xd.multilingual_politeness.PolitenessDataset(split="test"), 
     xft.WordGroups(distinct=26, scaling=1.5), xd.multilingual_politeness.PolitenessFixScore(),
    xd.multilingual_politeness.preprocess_politeness),
    (xd.emotion.EmotionDataset(split="test"), 
     xft.WordGroups(distinct=4, scaling=1.5), xd.emotion.EmotionFixScore(),
    xd.emotion.preprocess_emotion)
]

In [ ]:
from collections import defaultdict
from torch.utils.data import Dataset, Subset, DataLoader
from tqdm.auto import tqdm

batch_size = 1
shuffle = True

device = "cuda" if torch.cuda.is_available() else "cpu"

all_scores = []

for dataset, feature_extractor, metric, preprocess in l:
    torch.manual_seed(1234)
    if hasattr(dataset, 'get_dataloader'): # if the dataset has its own dataloader, then use that
        dataloader = dataset.get_dataloader(batch_size=batch_size, shuffle=shuffle)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    feature_extractor = feature_extractor.to(device)
    feature_extractor.eval()
    
    scores = []
    for batch in tqdm(dataloader):
        # Preprocess the batch using the preprocess function for that dataset
        X, metric_inputs = preprocess(batch)
        # move inputs to device if they are tensors
        X = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in X.items()}
        metric_inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in metric_inputs.items()}
        
        # extract expert features
        expert_feats = feature_extractor(**X)
        # evaluate metric scores
        score = metric(expert_feats, **metric_inputs)
        
        if not isinstance(score, torch.Tensor):
            score = torch.tensor(score)
        scores.append(score)
        if len(scores) > 5: # comment out if evaluating on all the data
            break
    all_scores.append(torch.stack(scores).view(-1).tolist())

In [ ]:
all_scores

In [ ]:
# get mean and std of each baseline

import numpy as np
from exlib.utils import bootstrap

for i in range(len(all_scores)):
    print(i, np.mean(all_scores[i]), bootstrap(torch.tensor(all_scores[i])).item())